# INMARSAT November Bill 2020

## Import Libraries

In [1]:
import pandas as pd

import numpy as np

import re

import os

## Cleaning Data: 
1. Uploading Invoice, 2. Removing NaNs, 3. Renaming Columns Needed, 4. Filtering Totals, 5. Resetting Index To Current Filtered Rows, 6. Removing 'Unamed' Columns, 7. Getting ID Numbers For New 'IDs' Column, 8. Getting Amounts Creating New 'Amounts Due' Column, 9. Turning Amounts Due from Strings of Text into Numbers, 10. Removing 'ID' and 'Totals' Column, 11. Saving Version to .csv, 12. Checking My Work, 13. Checking Amounts Due Totals

In [2]:
df = pd.read_csv('November_2020_Invoice.csv') #1. Uploading Invoice

df = df.fillna("") #2. Removing NaNs

df = df.rename(columns={"Unnamed: 8": "ID", "Unnamed: 19": "Totals"}) #3. Renaming Columns Needed

df = df.loc[df['ID'].str.contains('^to[a-z]*', flags=re.I, regex=True)] #4. Filtering Totals

df = df.reset_index(drop=True) #5. Resetting Index To Current Filtered Rows

df = df.loc[:, ~df.columns.str.contains('^Unnamed')] #6. Removing 'Unamed' Columns

def get_ID(ID):
    return ID.split(' ')[2] #7. Getting ID Numbers For New 'IDs' Column

df['IDs'] = df['ID'].apply(lambda x: get_ID(x)) #7. Getting ID Numbers For New 'IDs' Column

df['Amounts Due'] = df['Totals'].apply(lambda x: x.split('$')[1]) #8. Getting Amounts Creating New 'Amounts Due' Column

df['Amounts Due'] = pd.to_numeric(df['Amounts Due']) #9. Turning Amounts Due from Strings of Text into Numbers

df = df.drop(columns=['ID', 'Totals']) #10. Removing 'ID' and 'Totals' Column

df.to_excel('INMARSAT November 2020.xlsx', index=False) #11. Saving Version to CSV

df #12. Checking My Work

,Bill To:,IDs,Amounts Due
0,10/11/2019 -,000000000100,13.79
1,,000000000101,13.79
2,,000000000102,13.79
3,,000000000103,13.79
4,,000000000104,13.79
...,...,...,...
467,,881651461225,15.44
468,,881651477444,15.44
469,,881651477445,46.69
470,,881651477579,53.94


In [3]:
df.sum(axis = 0, skipna = True) #13. Checking Amounts Due Totals

Bill To:       10/11/2019 -10/11/2019 -10/11/2019 -10/11/2019...
IDs            0000000001000000000001010000000001020000000001...
Amounts Due                                                17825
dtype: object

## Merging .csv Files Based on ID Numbers
1. Defining Our Previous Notebook Saved, 2. PI and ID Number Data, 3. Uploading Data, 4. Printing to see our columns, 5. Merging Our Files Based on ID Number, 6. Saving Version to .csv, 7. Checking My Work, 8. Checking Amounts Due Totals

In [4]:
initial_workbook = 'INMARSAT November 2020.xlsx' #1. Defining Our Previous Notebook Saved
info_workbook = 'Active DF COMPLETE 16 Mar.xlsx' #2. Defining PI and ID Number Dataframe

df_initial = pd.read_excel(initial_workbook) #3. Uploading Data
df_info = pd.read_excel(info_workbook) #3. Uploading Data

print(df_initial.columns) #4. Printing to see our columns
print(df_info.columns) #4. Printing to see our columns


Index(['Bill To:', 'IDs', 'Amounts Due'], dtype='object')
Index(['IDs', 'ICCID', 'MSISDN-C', 'Type', 'PI'], dtype='object')


In [5]:
df_3 = pd.merge(df_initial, df_info[['IDs','PI','Type']], on='IDs', how='left') #5. Merging Our Files Based on ID Number
df_3.to_excel('INMARSAT November 2020 Data.xlsx', index=False) #6. Saving Version to .csv

df_3 #7. Checking My Work


,Bill To:,IDs,Amounts Due,PI,Type
0,10/11/2019 -,100,13.79,Sutton,SHORT BURST DATA
1,NaN,101,13.79,McPhaden,SHORT BURST DATA
2,NaN,102,13.79,McPhaden,SHORT BURST DATA
3,NaN,103,13.79,McPhaden,SHORT BURST DATA
4,NaN,104,13.79,McPhaden,SHORT BURST DATA
...,...,...,...,...,...
467,NaN,881651461225,15.44,McPhaden,RUDICS
468,NaN,881651477444,15.44,Sutton,RUDICS
469,NaN,881651477445,46.69,Sutton,RUDICS
470,NaN,881651477579,53.94,Sutton,RUDICS


In [6]:
df_3.sum(axis = 0, skipna = True) #8. Checking Amounts Due Totals

IDs                                              339427740230717
Amounts Due                                                17825
PI             SuttonMcPhadenMcPhadenMcPhadenMcPhadenMcPhaden...
Type           SHORT BURST DATASHORT BURST DATASHORT BURST DA...
dtype: object

## Generating PI and Type Billing Report
1. Upload Merged .csv File, 2. Rename Column, 3. Organizing Based on 'PI' and 'Type', 4. Removing Column, 5. Saving Billing Report to .csv, 6. Checking My Work, 7. Checking Amounts Due Totals

In [7]:
df_3 = pd.read_excel('INMARSAT November 2020 Data.xlsx') #1. Upload Merged .csv File

df_3 = df_3.rename(columns={"Amounts Due": "NOV'20 Amounts Due"}) #2. Rename Column

df_3 = df_3.groupby(['PI','Type','IDs']).sum() #3. Organizing Based on 'PI' and 'Type'

#df_3 = df_3.drop(columns=['IDs']) #4. Removing Column

df_3.to_excel('INMARSAT November 2020 BILL PI TYPES.xlsx') #5. Saving Billing Report to .csv

df_3 #6. Checking My Work

NOV'20 Amounts Due
PI     Type             IDs                             
Cronin RUDICS           881621467686               15.44
                        881621467706               15.44
                        881621467707               15.44
                        881631439111               58.19
                        881631439114               18.69
...                                                  ...
Sutton SHORT BURST DATA 127                        13.79
                        128                        13.79
                        129                        13.79
                        130                        13.79
                        137                        20.19

[472 rows x 1 columns]

In [8]:
df_3.sum(axis = 0, skipna = True) #7. Checking Amounts Due Totals

NOV'20 Amounts Due    17825.03
dtype: float64

## Generating PI Billing Report
1. Upload Merged .csv File, 2. Rename Column, 3. Organizing Based on 'PI', 4. Removing Column, 5. Saving Billing Report to .csv, 6. Checking My Work, 7. Checking Amounts Due Totals

In [9]:
df_3 = pd.read_excel('INMARSAT November 2020 Data.xlsx') #1. Upload Merged .csv File

df_3 = df_3.rename(columns={"Amounts Due": "2 NOV'20 Amounts Due"}) #2. Rename Column

df_3 = df_3.groupby(['PI']).sum() #3. Organizing Based on 'PI'

df_3 = df_3.drop(columns=['IDs']) #4. Removing Column

df_3.to_excel('INMARSAT November 2020 BILL PI.xlsx') #5. Saving Billing Report to .csv

df_3 #6. Checking My Work

,2 NOV'20 Amounts Due
PI,
Cronin,334.41
Cross,176.21
Dziak,63.43
Feely,164.19
Johnson,10409.71
Klinck,15.44
McPhaden,2612.08
Meinig - Dziak,46.32
Meinig - Feely,30.88


In [10]:
df_3.sum(axis = 0, skipna = True) #7. Checking Amounts Due Totals

2 NOV'20 Amounts Due    17825.03
dtype: float64